In [1]:
from dotenv import load_dotenv
import os

import ollama
import prompt
import requests
import runpod
import os

c:\Users\user\anaconda3\envs\Gameprj\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

runpod.api_key = os.getenv('RUNPOD_API_KEY')
pod_id = os.getenv("POD_ID")

In [3]:
import random

for i in range(10):
    print(random.randint(1, 3), end=' ')

2 3 1 1 3 2 3 3 2 3 

In [6]:
from pprint import pprint

RUNPOD_OLLAMA_URL = f"https://{pod_id}-11434.proxy.runpod.net"

def query_ollama(prompt: str, pod_id: str, model: str = "EEVE-Korean-10.8B") -> str:

    if not pod_id:
        raise ValueError("pod_id를 지정해주세요.")

    # RunPod SDK로 Pod에서 코드 실행
    client = ollama.Client(host=RUNPOD_OLLAMA_URL)

    response = client.chat(
        model=model,
        messages=[
            {
                'role': 'user',
                'content': '대한민국의 수도는 어디야? 짧게 대답해줘.',
            },
        ],
    )

    return response['message']['content']

In [7]:
response = query_ollama("안녕? 자기소개 해줘", pod_id)

_client.py          :1025 2025-11-02 23:35:18,908 HTTP Request: POST https://0jm09eziy6bkg7-11434.proxy.runpod.net/api/chat "HTTP/1.1 200 OK"


In [8]:
response

'대한민국(South Korea)의 수도는 서울입니다.'

In [ ]:
def calculate_similarity(description_ref: str, turn_cards: set):
    return list(turn_cards)[0]

In [ ]:
import random

def play_game(players: dict, image_text_set: set):
    """
    전체 게임 진행
    players: { "Player1": set(), "Player2": set(), ... }
    image_text_set: 남아있는 카드 set
    """
    turn_number = 1
    total_scores = {key: 0 for key in players.keys()}

    while True:
        print(f"\n=== 턴 {turn_number} 시작 ===")

        # 1. 플레이어 차례
        selected_card, description, players = player_turn("Player1", players)
        print(f"당신이 선택한 카드: {selected_card}")
        print(f"설명: {description}")

        # 2. AI 차례 (Player2~5)
        ai_selected_cards = []
        ai_related_cards_list = []

        for ai_index in range(2, 6):
            selected_ai_card, related_cards, players = ai_turn(ai_index, players)
            ai_selected_cards.append(selected_ai_card)
            ai_related_cards_list.extend([c for c in related_cards if c is not None])

        # 3. 이번 턴 카드 뭉치 set 구성 (총 5장)
        turn_cards = set([selected_card] + ai_selected_cards + ai_related_cards_list)
        print("이번 턴 카드:", turn_cards)

        # 4. 각 플레이어 set에서 턴 카드 제거 (중복 제거)
        for key in players.keys():
            players[key] -= turn_cards

        # 5. 투표 진행
        scores = {key: 0 for key in players.keys()}
        description_ref = description  # 사용자 설명을 기준으로 투표

        card_owner_map = {}
        for key, card_set in players.items():
            for card in card_set:
                card_owner_map[card] = key

        for key in players.keys():
            if key == "Player1":
                # 사용자 투표
                print("\n투표: 이번 턴 카드 중 가장 설명과 비슷한 카드 선택")
                turn_cards_list = list(turn_cards)
                for idx, card in enumerate(turn_cards_list):
                    print(f"{idx+1}: {card}")
                while True:
                    try:
                        vote = int(input(f"투표 카드 번호 선택 (1-{len(turn_cards_list)}): "))
                        if 1 <= vote <= len(turn_cards_list):
                            break
                        else:
                            print("범위를 벗어났습니다.")
                    except ValueError:
                        print("숫자를 입력하세요.")
                voted_card = turn_cards_list[vote-1]
            else:
                # AI 투표 (LLM 버전으로도 교체 가능)
                voted_card = calculate_similarity(description_ref, turn_cards)

            # 🔹 투표된 카드의 주인 찾기
            owner = card_owner_map.get(voted_card)

            # 🔹 주인에게 점수 1점 부여
            if owner:
                scores[owner] += 1


        # 6. 점수 누적
        for key in total_scores.keys():
            total_scores[key] += scores[key]

        print("이번 턴 점수:", scores)
        print("누적 점수:", total_scores)

        # 7. 턴 종료 후 뭉치 set에서 각 플레이어에게 카드 1장씩 추가 배부
        for key in players.keys():
            if image_text_set:
                new_card = image_text_set.pop()
                players[key].add(new_card)

        # 8. 뭉치 set이 비면 게임 종료
        if not image_text_set:
            print("\n뭉치 카드가 모두 사용되었습니다. 게임 종료!")
            break

        turn_number += 1

    return total_scores

# 사용 예시
# total_scores = play_game(players, image_text_set)
# print("최종 점수:", total_scores)
